# Importation les bibliothèques nécessaires

In [10]:
!pip install fuzzywuzzy
!pip install python-Levenshtein


In [11]:
# Importer les bibliothèques
import pandas as pd
from fuzzywuzzy import process
import folium

# Configurer l'affichage complet des colonnes dans pandas
pd.set_option("display.max_columns", None)

# Charger les données des 50 festivals les plus consultés dans la dernière semaine 

In [12]:
# Le fichier des festivals en France présente des problèmes : on l'ouvre et on cherche les problèmes en analysant les premières lignes
print("données des festivals en France")
france_festivals_file = "festivals_en_France.csv"
df_france = pd.read_csv(france_festivals_file, sep=";")
print("Aperçu des données Festivals en France :")
print(df_france.head())


données des festivals en France
Aperçu des données Festivals en France :
           ﻿Nom du festival Envergure territoriale  \
0          Festival andalou              Régionale   
1  Festival Rêves d'enfants                    NaN   
2      Printemps de Paroles                    NaN   
3                 Pharaonic                    NaN   
4           Sarcus Festival                    NaN   

  Région principale de déroulement Département principal de déroulement  \
0       Provence-Alpes-Côte d'Azur                             Vaucluse   
1       Provence-Alpes-Côte d'Azur                         Hautes-Alpes   
2                    Île-de-France                       Seine-et-Marne   
3             Auvergne-Rhône-Alpes                               Savoie   
4              Centre-Val de Loire                       Indre-et-Loire   

  Commune principale de déroulement  \
0                           Avignon   
1                          Briançon   
2                Bussy-Saint-Marti

In [13]:
#la colonne du nom des festivals dans le fichier festival_en_France.csv n'est pas reconnu
#On cherche son vrai nom puis on va le modifier 
print(df_france.columns.tolist())

['\ufeffNom du festival', 'Envergure territoriale', 'Région principale de déroulement', 'Département principal de déroulement', 'Commune principale de déroulement', 'Code postal (de la commune principale de déroulement)', 'Code Insee commune', 'Code Insee EPCI', 'Libellé EPCI', 'Numéro de voie', 'Type de voie (rue, Avenue, boulevard, etc.)', 'Nom de la voie', 'Adresse postale', "Complément d'adresse (facultatif)", 'Site internet du festival', 'Adresse e-mail', 'Décennie de création du festival', 'Année de création du festival', 'Discipline dominante', 'Sous-catégorie spectacle vivant', 'Sous-catégorie musique', 'Sous-catégorie Musique CNM', 'Sous-catégorie cinéma et audiovisuel', 'Sous-catégorie arts visuels et arts numériques', 'Sous-catégorie livre et littérature', 'Période principale de déroulement du festival', 'Identifiant Agence A', 'Identifiant', 'Géocodage xy', 'identifiant CNM']


In [14]:
# On modifie le nom de la première colonne
df_france.rename(columns={"\ufeffNom du festival": "Nom du festival"}, inplace=True)

In [15]:
# Nettoyer les colonnes des deux bases
df_top_50["Nom_clean"] = df_top_50["Nom"].str.lower().str.strip()
df_france["Nom_clean"] = df_france["Nom du festival"].str.lower().str.strip()
df_france["Commune_clean"] = df_france["Commune principale de déroulement"].str.lower().str.strip()

# Vérifier les premières lignes
print("Aperçu des données Festivals en France :")
print(df_france.head())

NameError: name 'df_top_50' is not defined

#  Extraire les coordonnées géographiques

In [49]:
df_france[["Latitude", "Longitude"]] = df_france["Géocodage xy"].str.split(",", expand=True)
df_france["Latitude"] = pd.to_numeric(df_france["Latitude"], errors="coerce")
df_france["Longitude"] = pd.to_numeric(df_france["Longitude"], errors="coerce")

# Fusion des données 

In [ ]:
# Rapprocher les noms des festivals avec un score de similarité
def match_festival(row, all_names):
    if pd.notna(row["Nom_clean"]):
        match, score = process.extractOne(row["Nom_clean"], all_names)
        if score > 80:  # Seuil de similarité
            return match
    return None

# Appliquer le rapprochement
df_france_names = df_france["Nom_clean"].dropna().tolist()
df_top_50["Match"] = df_top_50.apply(match_festival, all_names=df_france_names, axis=1)

# Fusionner avec le DataFrame France basé sur les correspondances
df_merged = pd.merge(df_france, df_top_50, left_on="Nom_clean", right_on="Match", how="outer")

# Vérifier les correspondances
print("Aperçu des données fusionnées :")
print(df_merged.head())

Aperçu des données fusionnées :
                      Nom du festival Envergure territoriale  \
0  1, 2, 3 … CINÉ ! FAIT SON FESTIVAL              Régionale   
1                  1, 2, 3, PESTACLES                    NaN   
2                  1, 2, 3… COMÉDIE !                    NaN   
3                           10 SUR 10                    NaN   
4                         100 DÉTOURS                    NaN   

  Région principale de déroulement Département principal de déroulement  \
0              Centre-Val de Loire                       Indre-et-Loire   
1               Nouvelle-Aquitaine                 Pyrénées-Atlantiques   
2       Provence-Alpes-Côte d'Azur                             Vaucluse   
3             Auvergne-Rhône-Alpes                         Haute-Savoie   
4             Auvergne-Rhône-Alpes                                Isère   

  Commune principale de déroulement  \
0                             Tours   
1                              Uzos   
2              

In [51]:
# Marquer les festivals populaires
df_merged["Populaire"] = ~df_merged["Classement"].isna()

# Supprimer les lignes sans coordonnées
df_merged = df_merged.dropna(subset=["Latitude", "Longitude"])


In [1]:
# Créer une carte centrée sur la France
map_festivals = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter des marqueurs pour les festivals
for _, row in df_merged.iterrows():
    if row["Populaire"]:  # Festivals populaires
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            popup=f"#{int(row['Classement'])} {row['Nom']} - {row['Commune principale de déroulement']}",
            icon=folium.Icon(color="red", icon="star"),
        ).add_to(map_festivals)
    else:  # Autres festivals
        folium.CircleMarker(
            location=[row["Latitude"], row["Longitude"]],
            radius=5,
            color="blue",
            fill=True,
            fill_color="blue",
            fill_opacity=0.6,
            popup=row["Nom du festival"],
        ).add_to(map_festivals)

# Sauvegarder la carte
map_festivals.save("carte_festivals_france.html")
print("Carte des festivals sauvegardée sous 'carte_festivals_france.html'")


NameError: name 'folium' is not defined

# Créer une carte avec Folium


#  Afficher la carte dans le notebook


#  Ouvrir la carte dans un navigateur
